In [1]:
# Import necessary libraries

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
from bs4 import BeautifulSoup
import requests
import lxml
import csv

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    conda-4.6.8                |           py36_0         876 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    openssl-1.1.1b             |       h14c3975_1         4.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0

The follow

In [2]:
# Web Scrapping data from the wikipedia page.

source = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(source)
soup = BeautifulSoup(page.content, 'lxml')

In [3]:
my_table = soup.find('table', class_='wikitable')
my_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [4]:
# Fetching the Table Data('td') and assigning it to a variable.
table_data = my_table.find_all('td')

In [5]:
# Creating column names and assigning it to a data frame.

column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
neighborhoods = pd.DataFrame(columns=column_names)
column_names

['PostalCode', 'Borough', 'Neighborhood']

In [6]:
# Creating a table after web scrapping.

tableSoup = soup.find_all("table")
table = tableSoup[0]
rows = table.find_all("tr")
for row in rows:
    columns = row.find_all("td")
    headers = row.find_all("th")
    
    if len(columns) == 0 : continue
    postcode = columns[0].text
    borough = columns[1].text
    neigh = columns[2].text.split("\n")[0]
    neighborhoods = neighborhoods.append({'PostalCode': postcode,
                          'Borough': borough,
                          'Neighborhood': neigh}, ignore_index=True)
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
df_neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains("[^Not assigned]")==True]
df_neighborhoods.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [8]:
df_neighborhoods['Neighborhood'].replace(to_replace="Not assigned", value=df_neighborhoods['Borough'], inplace=True)
df_neighborhoods.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [9]:
df_neigh = pd.DataFrame({'Neighborhood' : df_neighborhoods.groupby(by = ['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))}).reset_index()
df_neigh.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
# Importing a csv file that has the geographical coordinates of each postal code.

df_coordinates  = pd.read_csv('http://cocl.us/Geospatial_data')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# Merging the 2 data frame from a common point, the common point being 'Postal code' 

merge_data = (pd.merge(df_neigh, df_coordinates, left_on="PostalCode", right_on="Postal Code").
            drop('Postal Code', axis=1))
merge_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
merge_data = merge_data[merge_data['Borough'].str.contains('Toronto')].reset_index(drop=True)
merge_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [13]:
# Using Geolocator to get the geograpical coordinates of Toronto,Ontario.
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(merge_data['Latitude'], merge_data['Longitude'], merge_data['Neighborhood']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [16]:
# CREATING A FUCNTION TO REPEAT EXPLORING THE NEIGHBOTHOODS IN TORONTO
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                 'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [17]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
CLIENT_ID = 'WDQ40UWY10SDO3AEIOD214LY1V2XB25FOZRFOVRPE2VX4TGQ'
CLIENT_SECRET = 'HC2WEH3GKEJTFBWIXQ1U0ZOLH4XCRFCUNAY22MUCUCH2MLGC'
VERSION = '20180604'

In [21]:
toronto_venues_restaurant = getNearbyVenues(names=merge_data['Neighborhood'], latitudes=merge_data['Latitude'], longitudes=merge_data['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d157941735')
toronto_venues_restaurant.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Danforth West, Riverdale",43.679557,-79.352188,Globe Bistro,43.676680,-79.358155,New American Restaurant
1,"The Beaches West, India Bazaar",43.668999,-79.315572,Edward's 1290,43.663776,-79.328582,New American Restaurant
2,Studio District,43.659526,-79.340923,Rashers,43.660859,-79.341111,New American Restaurant
3,Davisville North,43.712751,-79.390197,JOEY,43.704311,-79.388512,New American Restaurant
4,Davisville North,43.712751,-79.390197,The Belsize Public House,43.702517,-79.387864,New American Restaurant


In [22]:
toronto_venues_restaurant.shape

(107, 7)

In [25]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [26]:
map_toronto_restaurant = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(toronto_venues_restaurant, 'red', map_toronto_restaurant)
map_toronto_restaurant

In [27]:
toronto_venues_highschools = getNearbyVenues(names=merge_data['Neighborhood'], latitudes=merge_data['Latitude'], longitudes=merge_data['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d13d941735')
toronto_venues_highschools.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Malvern Collegiate,43.683360,-79.293039,High School
1,The Beaches,43.676357,-79.293031,Neil McNeil Catholic High School,43.679723,-79.283146,High School
2,"The Danforth West, Riverdale",43.679557,-79.352188,Montcrest School,43.673748,-79.356864,High School
3,"The Danforth West, Riverdale",43.679557,-79.352188,City Adult Learning Centre,43.675408,-79.359885,High School
4,"The Danforth West, Riverdale",43.679557,-79.352188,Degrassi High School,43.684306,-79.349092,High School


In [28]:
toronto_venues_highschools.shape

(188, 7)

In [29]:
map_toronto_highschools = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(toronto_venues_highschools, 'green', map_toronto_highschools)
map_toronto_highschools

In [30]:
toronto_venues_university = getNearbyVenues(names=merge_data['Neighborhood'], latitudes=merge_data['Latitude'], longitudes=merge_data['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ae941735')
toronto_venues_university.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Beaches West, India Bazaar",43.668999,-79.315572,The Hamilton Institute For Recreational Studies,43.671802,-79.310166,University
1,Lawrence Park,43.728020,-79.388790,"Glendon Campus, York University",43.726981,-79.394296,University
2,Davisville North,43.712751,-79.390197,King George International College,43.708185,-79.394054,University
3,Davisville North,43.712751,-79.390197,London School of Business and Finance (LSBF),43.706718,-79.400255,General College & University
4,North Toronto West,43.715383,-79.405678,London School of Business and Finance (LSBF),43.706718,-79.400255,General College & University


In [31]:
toronto_venues_university.shape

(500, 7)

In [32]:
map_toronto_university = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(toronto_venues_university, 'purple', map_toronto_university)
map_toronto_university

In [34]:
toronto_venues_office = getNearbyVenues(names=merge_data['Neighborhood'], latitudes=merge_data['Latitude'], longitudes=merge_data['Longitude'], radius=1000, categoryIds='56aa371be4b08b9a8d573517')
toronto_venues_office.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Studio District,43.659526,-79.340923,That Toronto Studio,43.662936,-79.341073,Business Center
1,Studio District,43.659526,-79.340923,Moss Led,43.659042,-79.353753,Business Center
2,Studio District,43.659526,-79.340923,studio la beauté,43.664138,-79.343553,Business Center
3,Davisville North,43.712751,-79.390197,Ig Beauty,43.711315,-79.377740,Business Center
4,"Moore Park, Summerhill East",43.689574,-79.383160,Alternativa Canada,43.688661,-79.390861,Business Center


In [35]:
toronto_venues_office.shape

(121, 7)

In [36]:
map_toronto_office = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(toronto_venues_office, 'orange', map_toronto_office)
map_toronto_office

In [37]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0
            
            
            
            
            
merge_data.rename(columns={'Borough':'Neighbourhood'}, inplace=True)
addColumn(merge_data, 'Restaurant', toronto_venues_restaurant)
addColumn(merge_data, 'High Schools', toronto_venues_highschools)
addColumn(merge_data, 'Universities', toronto_venues_university)
addColumn(merge_data, 'Offices', toronto_venues_office)
merge_data

------------------------------------------------------------------------------
df_data = df3.copy()
df_data.rename(columns={'Localidades':'Localidad'}, inplace=True)
addColumn(df_data, 'Burger', bogota_venues_burger)
addColumn(df_data, 'High Schools', bogota_venues_highschools)
addColumn(df_data, 'Universities', bogota_venues_uni)
addColumn(df_data, 'Offices', bogota_venues_office)
df_data



In [39]:
merge_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [45]:
merge_data.drop(['PostalCode','Neighborhood'], axis=1)

,Borough,Latitude,Longitude
0,East Toronto,43.676357,-79.293031
1,East Toronto,43.679557,-79.352188
2,East Toronto,43.668999,-79.315572
3,East Toronto,43.659526,-79.340923
4,Central Toronto,43.728020,-79.388790
5,Central Toronto,43.712751,-79.390197
6,Central Toronto,43.715383,-79.405678
7,Central Toronto,43.704324,-79.388790
8,Central Toronto,43.689574,-79.383160
9,Central Toronto,43.686412,-79.400049


In [46]:
merge_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [47]:
merge_data.rename(columns={'Borough':'Neighbourhood'}, inplace=True)
addColumn(merge_data, 'Restaurant', toronto_venues_restaurant)
addColumn(merge_data, 'High Schools', toronto_venues_highschools)
addColumn(merge_data, 'Universities', toronto_venues_university)
addColumn(merge_data, 'Offices', toronto_venues_office)
merge_data

,PostalCode,Neighbourhood,Neighborhood,Latitude,Longitude,Restaurant,High Schools,Universities,Offices
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,2.0,0.0,0.0
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1.0,4.0,0.0,0.0
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1.0,0.0,1.0,0.0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1.0,2.0,0.0,3.0
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.0,5.0,1.0,0.0
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2.0,5.0,2.0,1.0
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0.0,10.0,2.0,0.0
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,2.0,6.0,4.0,0.0
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1.0,0.0,2.0,1.0
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,1.0,3.0,1.0,1.0


In [48]:
# positive weight because employees are even better customers
weight_offices = 2

# positive weight, because uni students are good customers
weight_universities = 1.5

# positive weight, because high school students are good customers
weight_schools = 1

# negative weight, because Carol wants to open a burger joint and thus wants to avoid concurrence as much as possible
weight_restaurant = -1


In [49]:
df_weight_data = merge_data[['Neighborhood']].copy()

In [51]:
df_weight_data['Score'] = merge_data['Restaurant'] * weight_restaurant + merge_data['High Schools'] * weight_schools + merge_data['Universities'] * weight_universities + merge_data['Offices'] * weight_offices
df_weight_data = df_weight_data.sort_values(by=['Score'], ascending=False)
df_weight_data

,Neighborhood,Score
17,Central Bay Street,97.5
14,"Ryerson, Garden District",94.0
18,"Adelaide, King, Richmond",86.0
12,Church and Wellesley,81.5
25,"Harbord, University of Toronto",80.5
29,"First Canadian Place, Underground city",70.0
20,"Design Exchange, Toronto Dominion Centre",67.5
15,St. James Town,67.5
21,"Commerce Court, Victoria Hotel",67.0
26,"Chinatown, Grange Park, Kensington Market",63.0


In [54]:
map_toronto_result = folium.Map(location=[latitude, longitude], zoom_start=15)

tor_win = merge_data[merge_data['Neighborhood'] == 'Central Bay Street']

for lat, lng, local in zip(bog_win['Latitude'], bog_win['Longitude'], tor_win['Neighborhood']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_toronto_result) 

addToMap(toronto_venues_restaurant[toronto_venues_restaurant['Neighborhood'] == 'Central Bay Street'], 'red', map_toronto_result)
addToMap(toronto_venues_highschools[toronto_venues_highschools['Neighborhood'] == 'Central Bay Street'], 'green', map_toronto_result)
addToMap(toronto_venues_university[toronto_venues_university['Neighborhood'] == 'Central Bay Street'], 'gold', map_toronto_result)
addToMap(toronto_venues_office[toronto_venues_office['Neighborhood'] == 'Central Bay Street'], 'fuchsia', map_toronto_result)

map_toronto_result